La función `AI_CLASSIFY` clasifica texto o imágenes en las categorías que les proporcionemos.

### Sintaxis

```sql
AI_CLASSIFY( <input> , <list_of_categories> [, <config_object>] )
```

### Argumentos:

**`<input>`**: Texto, imagen u objeto a clasificar. Para la clasificación de texto, el texto de entrada distingue entre mayúsculas y minúsculas. Es posible que obtengamos resultados diferentes para un mismo texto que utilice distintas mayúsculas y minúsculas.

**`<list_of_categories>`**: Array que contiene las categorías. Debe contener al menos una y como máximo 500 categorías únicas. Las categorías distinguen entre mayúsculas y minúsculas. 

Las categorías pueden ser strings simples u objetos SQL del mismo tipo. Si utilizamos objetos, podemos proporcionar una descripción para una o más categorías para mejorar la precisión de la clasificación.

Para cada categoría, debemos especificar lo siguiente:

- `label` (obligatorio): El nombre de la categoría.
- `description` (opcional): Describe la categoría en un máximo de 25 palabras.

**Opcional:**

**`<config_object>:`** Opciones de configuración especificadas como pares clave-valor. Claves compatibles:

- `task_description`: Una explicación de la tarea de clasificación de 50 palabras o menos. Esto puede ayudar al modelo a comprender el contexto de la tarea de clasificación y mejorar la precisión.
- `output_mode`: Se establece en `'multi'` para la clasificación multietiqueta. El valor predeterminado es `'single'` para la clasificación de una sola etiqueta.
- `examples`: Una lista de objetos de ejemplo para que el aprendizaje sea con pocos intentos. Cada ejemplo debe incluir:
    - `input`: Texto de ejemplo para clasificar.
    - `labels`: Lista de categorías correctas para la entrada.
    - `explanation`: Explicación de por qué la entrada se asigna a esas categorías.


## **`AI_CLASSIFY`: Texto**

### Ejemplo con los argumentos requeridos

Este ejemplo ilustra cómo utilizar la función `AI_CLASSIFY` solo con los argumentos requeridos.

In [ ]:
SELECT AI_CLASSIFY('El atardecer en la playa es un espectáculo natural que inspira paz y reflexión. Los tonos dorados, rosados y morados del cielo se funden con el sonido de las olas. Cada puesta de sol es única, regalando un momento efímero de belleza.', ['Naturaleza', 'Bienestar emocional', 'Fotografía']);

### Notas de uso

Para un rendimiento óptimo, siga estas pautas:

- Utilice texto en inglés simple para la entrada y las categorías.
- Limite la cantidad de texto que no esté en inglés simple en el texto de entrada. Por ejemplo, intente limitar el contenido como fragmentos de código o registros en la entrada de texto.
- El texto no debe contener código ni formatos que no sean de código abierto.
- No utilice abreviaturas, caracteres especiales ni jerga en las etiquetas de las categorías.
- Las categorías deben ser descriptivas. Por ejemplo, usar una categoría como Xa4s3 o la categoría 1 no producirá buenos resultados.
- Las categorías deben ser mutuamente excluyentes.
- Añadir una descripción clara de la tarea puede mejorar la precisión cuando la relación entre el texto de entrada y las categorías es ambigua o tiene matices.
- Añadir descripciones de las etiquetas puede mejorar la precisión en casos donde las descripciones son ambiguas o cuando se debe seguir una lógica específica al seleccionar una etiqueta en particular. Al escribir descripciones, concéntrese en los aspectos clave que distinguen una etiqueta de las demás.
- Los ejemplos pueden ayudar a mejorar la precisión.

### Ejemplo con una tabla de valores

El siguiente ejemplo crea una tabla, `clasificacion_textos`, que contiene una columna para texto y otra para posibles categorías del texto. Se llama a la función `AI_CLASSIFY` en cada fila de la tabla para clasificar el texto.

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

-- Crear la tabla
CREATE TABLE clasificacion_textos (
   texto VARCHAR(500),
   categorias ARRAY
);

In [ ]:
-- Insertar 5 filas con diferentes temas usando ARRAY_CONSTRUCT con la sintaxis SELECT

INSERT INTO clasificacion_textos (texto, categorias)
  SELECT 'El nuevo sistema de transporte público ha reducido significativamente el tráfico en la ciudad. Los tiempos de viaje han mejorado en un 30% desde su implementación.',
         ARRAY_CONSTRUCT('transporte','urbanismo','medio ambiente','tecnología');

INSERT INTO clasificacion_textos (texto, categorias)
  SELECT 'Los glaciares del Ártico continúan derritiéndose a un ritmo alarmante. Los científicos advierten que esto podría tener consecuencias irreversibles para los ecosistemas marinos.',
         ARRAY_CONSTRUCT('cambio climático','medio ambiente','ciencia','geografía');

INSERT INTO clasificacion_textos (texto, categorias)
  SELECT 'El reciente lanzamiento del videojuego ha batido todos los récords de ventas del sector. Los críticos lo consideran una obra maestra de la narrativa interactiva.',
         ARRAY_CONSTRUCT('entretenimiento','tecnología','videojuegos','economía');

INSERT INTO clasificacion_textos (texto, categorias)
  SELECT 'La nueva política fiscal ha generado opiniones divididas entre economistas. Los mercados han reaccionado con cautela ante los cambios propuestos.',
         ARRAY_CONSTRUCT('economía','política','finanzas','legislación');

INSERT INTO clasificacion_textos (texto, categorias)
  SELECT 'El festival de cine independiente presentó obras de realizadores de más de 25 países. La película ganadora aborda temas de identidad cultural en un contexto globalizado.',
         ARRAY_CONSTRUCT('arte','cultura','entretenimiento','sociedad');

In [ ]:
SELECT * FROM clasificacion_textos;

In [ ]:
SELECT texto,
       categorias,
       AI_CLASSIFY(texto, categorias)['labels']::string as clasificacion
FROM clasificacion_textos;

El siguiente ejemplo utiliza una clasificación de múltiples etiquetas.

In [ ]:
SELECT AI_CLASSIFY('El científico publicó un estudio revolucionario sobre el cambio climático que podría transformar las políticas energéticas mundiales.', ['deportes', 'ciencia y tecnología', 'política y sociedad', 'entretenimiento'], {'output_mode': 'multi'});

El siguiente ejemplo incluye una descripción de la tarea y descripciones de etiquetas.

In [ ]:
SELECT AI_CLASSIFY(
  'La actriz ganadora del Oscar anunció que donará el 50% de sus ganancias de la nueva película para construir escuelas en comunidades rurales.',
  [
      {'label': 'salud y medicina', 'description': 'Determina temas relacionados con la salud y la medicina'},
      {'label': 'entretenimiento y espectáculos'},
      {'label': 'educación y sociedad'},
      {'label': 'tecnología'}
  ],
  {
    'task_description': 'Devuelve la clasificación del texto proporcionado',
    'output_mode': 'multi'
  }
);

## **`AI_CLASSIFY`: Imágenes**

### Usando como entrada un archivo único

En este ejemplo vamos a clasificar una sola imagen que se encuentre en un stage.

In [ ]:
SELECT AI_CLASSIFY(TO_FILE('@imagenes', 'soumaya.jpg'), ['estadio', 'parque', 'comida', 'museo']):labels;

En el siguiente ejemplo clasificaremos todas las imágenes que se encuentran en la tabla `image_table` que previamente hemos creado.

In [ ]:
use database curso_ia;
use schema seccion_2;

SELECT
*,
AI_CLASSIFY(img, ['animal', 'auto', 'economía', 'museo']):labels AS clasificacion
FROM image_table;

### Usando en la solicitud un objeto construido por `PROMPT()`



In [ ]:
use database curso_ia;
use schema seccion_2;

SELECT
*,
AI_CLASSIFY(PROMPT('Clasifica la imagen {0}', img),
  ['animal', 'auto', 'economía', 'museo']):labels AS clasificacion
FROM image_table;

## Limitaciones

- Las funciones de Snowflake AI no admiten la actualización incremental de tablas dinámicas.
- Las funciones de Snowflake AI no funcionan con objetos `FILE` creados a partir de archivos en los siguientes tipos de stages:
    - Internal stage con modo de cifrado `TYPE = 'SNOWFLAKE_FULL'`
    - External stage con cualquier modo de cifrado del lado del cliente:
        - `TYPE = 'AWS_CSE'`
        - `TYPE = 'AZURE_CSE'`
    - Stage de usuario
    - Stage de tabla
    - Stage con nombres entre comillas dobles